## Bronze: Dados Demográficos
- API do Banco Mundial com dados demográficos dos países, de 2000 a 2023
- Origem dos dados: https://wbdata.readthedocs.io/en/stable

In [0]:
%run "../1 - Setup/Ingestao_bibliotecas_padrao"


In [0]:
import wbdata

In [0]:
paises_validos = [
    "AFG","DEU","AGO","ATG","ARG","SAU","AUS","BHS","BGD","BRB","BHR","BLZ",
    "BEN","BOL","BRA","BGR","BEL","BIH","CPV","CMR","CAN","QAT","CHL","CHN",
    "CYP","SGP","COL","COM","COG","KOR","CRI","CIV","HRV","CUB","DNK","DMA","SLV",
    "ARE","ECU","SVK","ESP","USA","EST","PHL","FIN","FRA","GHA","GRD","GRC","GTM",
    "GUY","GNB","GNQ","HTI","HND","HKG","HUN","IND","IDN","IRQ","IRL",
    "IRN","ITA","JAM","JPN","JOR","LVA","LBR","LUX","LBN","MWI","MLT","MYS","MHL",
    "MRT","MNE","MOZ","MEX","NAM","NIC","NGA","NOR","NZL","OMN","PAN",
    "PRY","NLD","PER","POL","PRT","KEN","GBR","DOM","RUS","SEN","SLE","SWZ","SUR",
    "SWE","CHE","KNA","STP","VCT","THA","TWN","TZA","CZE","TGO","TTO","TUN","TUR",
    "URY","VEN","VNM","ZAF","AUT"
]

#  Indicadores World Bank ---
indicadores = {
    "SP.POP.TOTL": "Populacao_Total",
    "SP.URB.TOTL.IN.ZS": "Percentual_Urbanizacao",
    "NY.GDP.MKTP.CD": "PIB_Total",
    "NY.GDP.PCAP.CD": "PIB_per_Capita",
    "NY.GDP.MKTP.KD.ZG": "Crescimento_PIB"
}

start_year = datetime.datetime(2000,1,1)
end_date   = datetime.datetime(2023,12,31)


df = wbdata.get_dataframe(indicadores,
                          country=paises_validos,
                          data_date=(start_year, end_year), 
                          convert_date=False) 

#Tratamento de Erro (Adicional)
if df.empty:
    print("⚠️ Aviso: O DataFrame está vazio. Verifique a disponibilidade dos dados para os países e indicadores/anos selecionados.")
else:
    df = df.reset_index()
    df.rename(columns={"country": "Pais", "date": "Ano"}, inplace=True)
    print("✅ Dados coletados e processados com sucesso!")

In [0]:
df = add_metadata_cols(df)
display(df)

In [0]:
#Cria tabela gerenciada no Unity Catalog
spark.sql("""
CREATE TABLE IF NOT EXISTS vitivinicultura.bronze_dados_demograficos.dados_demograficos_banco_mundial
""")
# Escrita no formato Delta Lake
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("vitivinicultura.bronze_dados_demograficos.dados_demograficos_banco_mundial")

